compute the fisher information via its approximation as explained in  
"Piazzola, C., Tamellini, L., & Tempone, R. (2021). A note on tools for prediction under uncertainty and identifiability of SIR-like dynamical systems for epidemiology. Mathematical Biosciences, 332, 108514. https://doi.org/10.1016/J.MBS.2020.108514"

In [1]:
using Pkg
Pkg.activate("C:/Users/lisah/Documents/Repos/ma-code")

  Activating project at `C:\Users\lisah\Documents\Repos\ma-code`


In [2]:
using DifferentialEquations, CSV, DataFrames, Plots, Distributions, ForwardDiff, LinearAlgebra, Random

In [3]:
# functions that remain unchanged

function create_grid()
    a_vals = 0.0:0.1:2.0
    n_vals = 0.0:0.1:4.0

    grid = [(a, n) for n in n_vals, a in a_vals] 
    return grid
end

function randomize_data(df::DataFrame, noise::Float64)
    if noise == 0.0
        return df
    else
        Random.seed!(1) # make it reproducible
        df[!, "w"] .= df[!, "w"] .+ rand(Normal(0, noise), nrow(df))
        df[!, "n"] .= df[!, "n"] .+ rand(Normal(0, noise), nrow(df))
        return df
    end
end

function store_fish_data(w0::Float64,m::Float64,M::Int64,noise::Float64,df::DataFrame, path_to_repo::String)
    CSV.write("$(path_to_repo)fish_$(w0)_$(m)_$(M)_$(noise).csv", df)
end

store_fish_data (generic function with 1 method)

In [4]:
abstract type AbstractHyperprm end

struct Hyperprm <: AbstractHyperprm
    w0::Real
    n0::Real
    a::Real
    m::Real
    M::Real
    noise::Real
end

"""define the klausmeier model equations
"""
function klausmeier!(du,u,p,t)
    du[1] = -u[1] - u[1] * u[2]^2 + p[1] # water compartment
    du[2] = u[1] * u[2]^2 - p[2] * u[2] # biomass compartment
end

"""solves the klausmeier model for given set of parameters
"""
function sol_klausmeier(hprm::Hyperprm)
    u0 = [hprm.w0; hprm.n0]
    p = [hprm.a; hprm.m]
    tspan = (0.0, hprm.M-1)

    prob = ODEProblem(klausmeier!, u0, tspan, p)
    sol = solve(prob,
        saveat=1.0  # consider specific time points
    )

    return DataFrame(time=sol.t, w=sol[1, :], n=sol[2, :])
end

sol_klausmeier

In [5]:
function vectorize_sol(x, w0, m, M, noise)
    hprm = Hyperprm(w0, x[2], x[1], m, M, noise) #w0,n0,a,m,M
    sol = sol_klausmeier(hprm) # returns df
    return reduce(vcat, eachcol(select(sol, Not(:time)))) # returns vector [w,n]^T
end

vectorize_sol (generic function with 1 method)

In [6]:
path = "C:/Users/lisah/Documents/Repos/ma-code/data/fisher/m0.45v2/"
M_vals = [10, 100,500, 1000]
noise_vals = [0.0]
w0 = 0.95
m = 0.45

0.45

In [7]:
for M in M_vals
        for noise in noise_vals

            grid = create_grid()
            fish = zeros(41, 21)

            # evaluate fisher info on grid
            for i in range(1, 41)
                for j in range(1, 21)
                    pt = grid[i,j] # pt to differentiate in
                    eval_pt = [pt[1], pt[2]] # x = [a,n0]

                    # compute fisher information
                    J = ForwardDiff.jacobian(x -> vectorize_sol(x, w0, m, M, noise), eval_pt) # returns jacobian matrix
                    H = J' * J # approximation of FIM
                    FIM = H # in reference they use +
                    fish_val = tr(FIM)
                    fish[i,j] = fish_val
                end
            end
            
            #create data frame
            a_eval_pts = string.(0.0:0.1:2.0)
            df_fish = DataFrame(fish, a_eval_pts)

            store_fish_data(w0, m, M, noise, df_fish, path)
        end
    end